# Diet Optimization: Using CVXPY with cuOpt Backend

This notebook demonstrates how to use **CVXPY's intuitive modeling interface** with **NVIDIA cuOpt as the solver backend**. This gives you the best of both worlds:
- 🎯 CVXPY's clean, Pythonic syntax for modeling optimization problems
- 🚀 cuOpt's GPU-accelerated solving for high performance

## Problem Description

We'll solve the classic diet optimization problem:
- Minimize the cost of food purchases
- Meet nutritional requirements (calories, protein, fat, sodium)
- Satisfy additional constraints

This approach is ideal when you want to:
- Leverage CVXPY's familiar API for rapid prototyping
- Get GPU-accelerated performance from cuOpt
- Switch between different solver backends easily


## Environment Setup


In [ ]:
import subprocess
import html
from IPython.display import display, HTML

def check_gpu():
    try:
        result = subprocess.run(["nvidia-smi"], capture_output=True, text=True, timeout=5)
        result.check_returncode()
        lines = result.stdout.splitlines()
        gpu_info = lines[2] if len(lines) > 2 else "GPU detected"
        gpu_info_escaped = html.escape(gpu_info)
        display(HTML(f"""
        <div style="border:2px solid #4CAF50;padding:10px;border-radius:10px;background:#e8f5e9;">
            <h3>✅ GPU is enabled</h3>
            <pre>{gpu_info_escaped}</pre>
        </div>
        """))
        return True
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired, FileNotFoundError, IndexError) as e:
        display(HTML("""
        <div style="border:2px solid orange;padding:10px;border-radius:10px;background:#fff3e0;">
            <h3>⚠️ GPU not detected</h3>
            <p>cuOpt will still work but may not be GPU-accelerated.</p>
        </div>
        """))
        return False

check_gpu()


In [ ]:
!pip install cvxpy numpy pandas matplotlib

In [ ]:
# Uncomment to install required packages in Google Colab or similar environments
# !pip uninstall -y cuda-python cuda-bindings cuda-core

# For cu12
# !pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-cu12 nvidia-nvjitlink-cu12 rapids-logger==0.1.19

# For cu13
# !pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-cu13 nvidia-nvjitlink-cu13 rapids-logger==0.1.19


## Import Required Libraries


In [ ]:
import numpy as np
import pandas as pd
import time
import cvxpy as cp

print(f"CVXPY version: {cp.__version__}")
print("✓ Libraries imported successfully!")
print("\n💡 Key insight: CVXPY can use cuOpt as a backend solver via solver='CUOPT'")

## Problem Data

Define the diet optimization problem data based on USDA Dietary Guidelines.


In [ ]:
# Nutrition guidelines
categories = {
    "calories": {"min": 1800, "max": 2200},
    "protein": {"min": 91, "max": float('inf')},
    "fat": {"min": 0, "max": 65},
    "sodium": {"min": 0, "max": 1779}
}

# Food costs per serving (in dollars)
food_costs = {
    "hamburger": 2.49,
    "chicken": 2.89,
    "hot dog": 1.50,
    "fries": 1.89,
    "macaroni": 2.09,
    "pizza": 1.99,
    "salad": 2.49,
    "milk": 0.89,
    "ice cream": 1.59
}

# Nutrition values: [calories, protein, fat, sodium]
nutrition_data = {
    "hamburger": [410, 24, 26, 730],
    "chicken": [420, 32, 10, 1190],
    "hot dog": [560, 20, 32, 1800],
    "fries": [380, 4, 19, 270],
    "macaroni": [320, 12, 10, 930],
    "pizza": [320, 15, 12, 820],
    "salad": [320, 31, 12, 1230],
    "milk": [100, 8, 2.5, 125],
    "ice cream": [330, 8, 10, 180]
}

# Display as DataFrame
nutrition_df = pd.DataFrame(nutrition_data, index=list(categories.keys())).T
nutrition_df['Cost ($)'] = list(food_costs.values())
print("📊 Food Database:")
print(nutrition_df)


## Formulate Problem with CVXPY

Use CVXPY's intuitive syntax to model the optimization problem.


In [ ]:
# Create decision variables
food_names = list(food_costs.keys())
n_foods = len(food_names)

# x[i] = number of servings of food i to purchase (non-negative continuous variables)
x = cp.Variable(n_foods, nonneg=True, name="servings")

print(f"✓ Created {n_foods} decision variables")
print(f"  Variables represent servings of: {', '.join(food_names)}")


In [ ]:
# Define objective function: minimize total cost
cost_vector = np.array([food_costs[food] for food in food_names])
objective = cp.Minimize(cost_vector @ x)

print("✓ Objective function: Minimize", cost_vector @ x)
print(f"  Total cost = sum of (servings × cost per serving)")


In [ ]:
# Create nutrition matrix
# Rows = nutrients (calories, protein, fat, sodium)
# Columns = foods
nutrition_matrix = np.array([nutrition_data[food] for food in food_names]).T

print(f"✓ Nutrition matrix shape: {nutrition_matrix.shape}")
print(f"  {nutrition_matrix.shape[0]} nutrients × {nutrition_matrix.shape[1]} foods")


In [ ]:
# Define nutritional constraints
constraints = []

for i, (category, bounds) in enumerate(categories.items()):
    # Calculate total nutrition for this category
    nutrition_intake = nutrition_matrix[i, :] @ x
    
    # Lower bound constraint
    constraints.append(nutrition_intake >= bounds["min"])
    
    # Upper bound constraint (if not infinity)
    if bounds["max"] != float('inf'):
        constraints.append(nutrition_intake <= bounds["max"])

print(f"✓ Created {len(constraints)} nutritional constraints:")
for category, bounds in categories.items():
    if bounds["max"] == float('inf'):
        print(f"  - {category}: ≥ {bounds['min']}")
    else:
        print(f"  - {category}: [{bounds['min']}, {bounds['max']}]")


In [ ]:
# Create the CVXPY problem
problem = cp.Problem(objective, constraints)

print(f"\n✓ CVXPY Problem created:")
print(f"  - Variables: {problem.size_metrics.num_scalar_variables}")
print(f"  - Constraints: {len(constraints)}")
print(f"  - Problem type: {'LP' if not problem.is_mixed_integer() else 'MILP'}")


## Solve with cuOpt Backend

Now we solve the CVXPY problem using cuOpt as the backend solver.


In [ ]:
print("🚀 Solving with cuOpt backend...")
print("="*60)

start_time = time.time()

# Solve using cuOpt as the backend solver
# Using the most basic call - cuOpt will use its default settings
try:
    result = problem.solve(solver="CUOPT")
    solve_time = time.time() - start_time
    
    print("="*60)
    print(f"✅ Solved in {solve_time:.4f} seconds")
    print(f"📊 Status: {problem.status}")
    print(f"💰 Optimal cost: ${problem.value:.2f}")
except Exception as e:
    solve_time = time.time() - start_time
    print("="*60)
    print(f"❌ Error solving with cuOpt: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Trying with a different solver (SCS) as fallback...")
    try:
        result = problem.solve(solver="SCS")
        solve_time = time.time() - start_time
        print(f"✅ SCS solved in {solve_time:.4f} seconds")
        print(f"📊 Status: {problem.status}")
        print(f"💰 Optimal cost: ${problem.value:.2f}")
    except Exception as e2:
        print(f"❌ SCS also failed: {e2}")


## Analyze Solution


In [ ]:
if problem.status == cp.OPTIMAL:
    print("\n" + "="*60)
    print("📦 OPTIMAL FOOD PURCHASES")
    print("="*60)
    
    total_cost = 0
    purchase_data = []
    
    for i, food in enumerate(food_names):
        servings = x.value[i]
        if servings > 0.001:  # Only show significant purchases
            cost = servings * food_costs[food]
            total_cost += cost
            purchase_data.append({
                'Food': food,
                'Servings': f'{servings:.3f}',
                'Cost/Serving': f'${food_costs[food]:.2f}',
                'Total Cost': f'${cost:.2f}'
            })
            print(f"  {food:12s}: {servings:6.3f} servings @ ${food_costs[food]:.2f} = ${cost:6.2f}")
    
    print(f"\n  {'TOTAL COST':12s}: ${total_cost:6.2f}")
    
    # Create DataFrame
    purchase_df = pd.DataFrame(purchase_data)
    
else:
    print(f"\n⚠️  Problem status: {problem.status}")


In [ ]:
if problem.status == cp.OPTIMAL:
    print("\n" + "="*60)
    print("🍎 NUTRITIONAL INTAKE")
    print("="*60)
    
    nutrition_results = []
    
    for i, (category, bounds) in enumerate(categories.items()):
        total = nutrition_matrix[i, :] @ x.value
        min_val = bounds["min"]
        max_val = bounds["max"]
        
        # Check if constraints are satisfied
        min_satisfied = total >= min_val - 1e-6
        max_satisfied = (max_val == float('inf')) or (total <= max_val + 1e-6)
        status = "✓" if (min_satisfied and max_satisfied) else "✗"
        
        nutrition_results.append({
            'Nutrient': category.capitalize(),
            'Actual': f'{total:.1f}',
            'Min Required': f'{min_val:.0f}',
            'Max Allowed': 'None' if max_val == float('inf') else f'{max_val:.0f}',
            'Status': status
        })
        
        if max_val == float('inf'):
            print(f"  {category:10s}: {total:7.1f} (min: {min_val:5.0f}) {status}")
        else:
            print(f"  {category:10s}: {total:7.1f} (range: [{min_val:5.0f}, {max_val:5.0f}]) {status}")
    
    nutrition_df_results = pd.DataFrame(nutrition_results)


## Key Takeaways

### Using cuOpt through CVXPY

**Benefits:**
- 🎯 Use CVXPY's intuitive modeling syntax
- 🚀 Get GPU-accelerated performance from cuOpt
- 🔄 Easy to switch between different solvers

**How to use:**
```python
# Simply specify solver="CUOPT" when calling solve()
problem.solve(solver="CUOPT", verbose=True)
```

**Note on Solver Parameters:**
The cuOpt solver through CVXPY uses default parameters which work well for most problems. For advanced configuration (like `pdlp_solver_mode` or `solver_method`), you may need to use the cuOpt Python API directly as shown in the other notebooks in this folder.

### When to Use This Approach

**Use CVXPY + cuOpt when:**
- You're already familiar with CVXPY syntax
- You want to prototype quickly with a familiar interface
- You want GPU acceleration without changing your modeling code
- You need to easily compare different solvers

**Use cuOpt Python API directly when:**
- You need fine-grained control over solver settings
- You're building production systems
- You need to use advanced cuOpt features

### Conclusion

This notebook demonstrates that you can leverage cuOpt's GPU-accelerated solving power while using CVXPY's clean, Pythonic modeling interface. This gives you the flexibility to use familiar tools while still benefiting from high-performance GPU computation.
